In [ ]:
!pip install --upgrade google-cloud-aiplatform
!pip install google-cloud-secret-manager

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from vertexai.language_models import TextGenerationModel, ChatModel, InputOutputTextPair

import getpass
from google.colab import userdata
import base64
import vertexai
from typing import List, Optional
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.generative_models import GenerativeModel,GenerationConfig, Part, FinishReason, FunctionDeclaration,Tool,HarmBlockThreshold
from vertexai.generative_models import HarmCategory, grounding, GenerationResponse
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding as preview_grounding
from google.cloud import aiplatform

**GCP and Vertex AI auth setup**

In [ ]:

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "mysecret.json"
vertexai.init(project="myproject", location="us-central1")

**GEMINI CLIENT USAGE EXAMPLE**

In [ ]:


MODEL_ID = "gemini-1.5-flash-001"
####### without system prompt######
model = GenerativeModel(MODEL_ID)

####### with system prompt ############
model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "You are a helpful AI assistant",
        "Your mission is to help user with Alteon CLI commands",
    ],
)
########## Set model parameters ###############
generation_config = GenerationConfig(
    temperature=0.5, # LOW temp values of 0-0.2 recommeneded for rag and code generation.
    top_p=0.95,
    top_k=40,
    candidate_count=1, #The number of response variations to return. This value must be 1
    max_output_tokens=8192,
    ####### additional useful params #######
#    response_mime_type="application/json" # text/plain: (default)
#    stop_sequences = Optional: List[string] list of stop words. Max 5 items in list
)

# safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}


#reduced safety settings
lesssafesettings = {
#    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE, # only for monthly invoiced billed account
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

prompt = """
  User input: Can you help me with Alteon cli commands to configure a virtual server with ip address 44.3.2.3 ?
  Answer:
"""

response = model.generate_content(
    prompt,
    generation_config=generation_config,
    safety_settings=lesssafesettings,
#    stream=True  to turn on streaming
)

##### when stream is true use this to stream response####
#for resp in response:
#    print(resp.text, end="")


################# Response ######################

print ("Printing the model response attributes - text, usage_metadata, finish reason, safety ratings\n")
print(f"\nAnswer:\n{response.text}") # Return generated response


print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')
print(f"\nFinish reason:\n{response.candidates[0].finish_reason}")
print(f"\nSafety settings:\n{response.candidates[0].safety_ratings}")

print("Printing full response:\n")
print (response)



**Gemini Grouding API with function calling**

- add document to datastore and link to app in Vertex AI - > app builder.
- Uses function calling


In [ ]:

from vertexai.preview.generative_models import grounding as preview_grounding
from vertexai.generative_models import (
    GenerativeModel,
    GenerationResponse)


############ Setup DATASTORE ###########
### The data here contains syslogs

DATA_STORE_PROJECT_ID = PROJECT_ID
DATA_STORE_REGION = "global"
DATA_STORE_ID = "syslog_1721944246450"
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"

MODEL_ID = "gemini-1.5-flash-001"
model = GenerativeModel(
    MODEL_ID,
)

generation_config = GenerationConfig(
    temperature=0,
    top_p=0.95,
    top_k=40,
    candidate_count=1,
    max_output_tokens=8192,

)

lesssafetysettings = {
#    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE, # only for monthly invoiced billed account
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

prompt1 = "Show me all logs with cpu"
prompt2 = "how do i disable management access on data port ?"

#### instantiate grouding tool #######
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(preview_grounding.VertexAISearch(datastore=datastore))
)


response = model.generate_content(
    prompt2,
    tools=[tool],
    generation_config=generation_config,
    safety_settings=nosafetysettings,
)



def parse_print_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata
    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)
    print ("text bytes",text_bytes )
    for segment in grounding_metadata.grounding_supports:
      print (segment.segment, segment.confidence_scores)
    for chunk in grounding_metadata.grounding_chunks:
        print(chunk.retrieved_context.uri)
        print(chunk.retrieved_context.title)

print(parse_print_response(response))

text bytes b'To disable management access on a data port of a standalone appliance, use the command `/cfg/sys/access/port/rem <port number>`. To disable management access on a VLAN for a virtual ADC, use the command `/cfg/sys/access/vlan/rem <vlan number>`. \n'
end_index: 128
text: "To disable management access on a data port of a standalone appliance, use the command `/cfg/sys/access/port/rem <port number>`."
 []
start_index: 129
end_index: 244
text: "To disable management access on a VLAN for a virtual ADC, use the command `/cfg/sys/access/vlan/rem <vlan number>`."
 [0.9883381724357605]
gs://lab_data_1/userguides/Alteon CLI User Guide.pdf
Alteon CLI User Guide
None
